In [1]:
# 安裝套件
# !pip install -q accelerate==0.22.0 peft==0.4.0 bitsandbytes==0.41.1 transformers==4.31.0 trl==0.7.1

In [1]:
!pip install -q bitsandbytes==0.41.1 trl==0.7.1

In [2]:
# 匯入必要的模組和套件
import warnings  # 忽略警告

warnings.simplefilter("ignore")
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/matplotlib_inline.backend_inline'), WindowsPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=121, Highest Compute Capability: 8.6.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/blob/main/how_to_use_nonpytorch_cuda.md
CUDA SETUP: Loading binary c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\bitsandbytes\li

RuntimeError: 
        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:
# 匯入必要的模組和套件
import warnings  # 忽略警告

warnings.simplefilter("ignore")
import os
import torch

### **Hyperparameter**

model_name : 預訓練模型的名稱。

dataset_name : 訓練數據集的文件路徑。

new_model : 新模型的名稱。

lora_r, lora_alpha, lora_dropout : LoRA 的相關參數。

use_4bit : 是否使用 4 位定點數。

bnb_4bit_compute_dtype : 4 位定點數計算的數據類型。

bnb_4bit_quant_type : 4 位定點數的量化類型。

use_nested_quant : 是否使用嵌套量化。

output_dir : 輸出的目錄。

num_train_epochs : 訓練的週期數。

fp16 : 是否使用 16 位浮點數。

bf16 : 是否使用 bfloat16。

per_device_train_batch_size : 每個設備的訓練批次大小。

per_device_eval_batch_size : 每個設備的評估批次大小。

gradient_accumulation_steps : 梯度累積的步數。

gradient_checkpointing : 是否使用梯度檢查點。

max_grad_norm : 梯度的最大範數。

learning_rate : 學習速率。

weight_decay : 權重衰減。
optim : 優化器。

lr_scheduler_type : 學習速率的調整方式。

max_steps : 最大的訓練步數。

warmup_ratio : 學習速率的熱身比例。

group_by_length : 是否根據句子的長度將它們分組。

save_steps : 模型保存的步數。

logging_steps : 日誌記錄的步數。

max_seq_length : 輸入序列的最大長度。

packing : 是否打包序列。

device_map : 使用哪一個GPU。

詳細參數：https://huggingface.co/docs/transformers/v4.31.0/en/main_classes/trainer#transformers.TrainingArguments

### **載入模型及設定參數**

In [ ]:
model_name = "huangyt/Taiwan-LLaMa-v1.0-ccp2-r16-q_k_v"
new_model = "llama-2-7b-custom"
dataset_name = "./prompt_n_response.jsonl"

# Quantized LLMs with Low-Rank Adapters (QLoRA) parameters

lora_r = 64
lora_alpha = 16
lora_dropout = 0.1


# bitsandbytes parameters 輕量級封裝，專門用於CUDA自定義函數，特別是8位優化器、矩陣乘法和量化

use_4bit = True
bnb_4bit_compute_dtype = "bfloat16"  # float16 or bfloat16
bnb_4bit_quant_type = "nf4"  # fp4 or nf4
use_nested_quant = False


# TrainingArguments parameters

output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 5
logging_steps = 5


# Supervised finetuning (SFT) parameters

max_seq_length = None
packing = False
device_map = {"": 0}  # {"": 0} or "auto"
# device_map = "auto"

### **下載 GPT4 生成的資料**

In [ ]:
# # 下載訓練及測試資料集
# !wget https://github.com/f901107/Fine_tuning_LLMs/releases/download/Dataset/train.jsonl
# !wget https://github.com/f901107/Fine_tuning_LLMs/releases/download/Dataset/test.jsonl

### **讀取資料集和前處理**

In [ ]:
# 讀取資料集
train_dataset = load_dataset(
    "json", data_files="./prompt_n_response.jsonl", split="train", 
)  # 從JSON文件中載入訓練數據集
valid_dataset = load_dataset(
    "json", data_files="./prompt_n_response.jsonl", split="train"
)  # 從JSON文件中載入驗證數據集

# 對數據集進行前處理，將提示和回應組合成文本對
train_dataset = train_dataset.map(
    lambda examples: {
        "text": [
            prompt + response
            for prompt, response in zip(examples["prompt"], examples["response"])
        ]
    },
    batched=True,
)
valid_dataset = valid_dataset.map(
    lambda examples: {
        "text": [
            prompt + response
            for prompt, response in zip(examples["prompt"], examples["response"])
        ]
    },
    batched=True,
)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

### **訓練模型**

In [2]:

!nvidia-smi

Wed Feb 28 18:19:30 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.61                 Driver Version: 551.61         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   37C    P8             41W /  390W |    1244MiB /  24576MiB |     27%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch

print(torch.cuda.is_available())

True


In [4]:
torch.cuda.get_device_capability()

(8, 6)

In [ ]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("text-generation", model="yentinglin/Taiwan-LLM-7B-v2.1-chat")

In [ ]:
pip install -q auto_gptq

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# from transformers import AutoTokenizer,TextStreamer,TextIteratorStreamer, AutoModel
# from auto_gptq import AutoGPTQForCausalLM


# tokenizer = AutoTokenizer.from_pretrained("weiren119/Taiwan-LLaMa-v1.0-4bits-GPTQ")
# model = AutoModel.from_pretrained("weiren119/Taiwan-LLaMa-v1.0-4bits-GPTQ")


# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("huangyt/Taiwan-LLaMa-v1.0-ccp2-r16-q_k_v")
# model = AutoModelForCausalLM.from_pretrained("huangyt/Taiwan-LLaMa-v1.0-ccp2-r16-q_k_v")



CUDA extension not installed.
CUDA extension not installed.


In [ ]:
# pip install bitsandbytes

  Using cached bitsandbytes-0.42.0-py3-none-any.whl.metadata (9.9 kB)
Using cached bitsandbytes-0.42.0-py3-none-any.whl (105.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import AutoTokenizer,TextStreamer,TextIteratorStreamer, AutoModel, AutoModelForCausalLM

# 定義位元和字節量化的相關配置
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# 檢查 GPU 是否與 bfloat16 相容
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# 從預訓練模型中載入自動生成模型
model = AutoModelForCausalLM.from_pretrained(
    model_name, quantization_config=bnb_config, device_map=device_map
)
model.config.use_cache = False
model.load_in_8bit_fp32_cpu_offload = True
model.config.pretraining_tp = 1

# 載入與模型對應的分詞器
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training




config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
# 定義 Prompt Engineering Fine-Tuning （PEFT）的相關設定
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# 設置訓練參數
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",  # "all"
    evaluation_strategy="steps",
    eval_steps=5,  # 每5步驗證
    
)

# 使用 SFTTrainer 進行監督式微調訓練
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,  # 在這裡傳入驗證集
    eval_dataset=valid_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# 開始訓練模型
trainer.train()

# 儲存微調後的模型
trainer.model.save_pretrained(new_model)




  8%|▊         | 4/49 [17:25<3:00:36, 240.82s/it]

### **輸出日誌結果**

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs
# 日誌輸出
logging.set_verbosity(logging.CRITICAL)

### **生成結果**

In [ ]:
# 執行模型的文本生成流程
prompt = "What is a large language model?"
pipe = pipeline(
    task="text-generation", model=model, tokenizer=tokenizer, max_length=200
)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]["generated_text"])  # 輸出生成的文本

### **清理 VRAM**

In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc  # 清理垃圾桶

gc.collect()
gc.collect()

### **將模型存入 Google Drive 中**

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

model_path = "/content/drive/MyDrive/GPT_Llama-2_fine-tune"  # 更改為您的路徑

# 以FP16重新載入模型並將其與LoRA權重合併
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# 重新載入分詞器以進行保存
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 儲存合併後的模型
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

### **從 Google Drive 載入微調的模型並進行推論**

In [ ]:
from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

drive.mount("/content/drive")

model_path = "/content/drive/MyDrive/GPT_Llama-2_fine-tune"  # 更改為您儲存模型的路徑

model = AutoModelForCausalLM.from_pretrained(
    model_path, device_map="auto", offload_folder="offload", torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

### **推論**

In [ ]:
prompt = "What is 2 + 2?"  # 更改為期望的提示
gen = pipeline("text-generation", model=model, max_new_tokens=2048, tokenizer=tokenizer)
result = gen(prompt)
print(result[0]["generated_text"])